# Evaluation Retrival

Um ganze sätze zu prüfen und zu vergleichen wird ebenfalls ein Cosinus-Similarity genutzt. Hierzu werden die Sätze in Vektoren umgewandelt und dann verglichen.

Die Testdaten wurden von Claude Code anhand der Produktdatei generiert. Sinnvoll wäre es, mit dem Kunden zu sprechen und echte Fragen zu sammeln.

prdukt_chunks muss so ;)

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import json
import chromadb

client = chromadb.PersistentClient(path="../3-indexing/chroma_db")
collection = client.get_collection('prdukt_chunks')
model = SentenceTransformer('deepset/gbert-large')

## Queries

Alle Queries wurden mit einem LLM erstellt, die Kategorien sind von mir.

In [ ]:
with open('test_queries.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

all_queries = []
for query_obj in test_data['queries']['technical_specs']:
    all_queries.append(query_obj['query'])

print(f"Geladene Queries: {len(all_queries)}")

## Retrieval

In [ ]:
def retrieve(queries, top_k=10):
    
    queries_embedded = model.encode(queries, normalize_embeddings=True, show_progress_bar=True)
    results = []
    
    for i, query in enumerate(queries):

        result = collection.query(
            query_embeddings=[queries_embedded[i].tolist()], 
            n_results=top_k
        )

        for rank, (doc, dist, meta) in enumerate(
            zip(
                result['documents'][0], 
                result['distances'][0],
                result['metadatas'][0]
            ), 
            start=1
        ):
            results.append({
                'query': query,
                'rank': rank,
                'document': doc,
                'distance': dist,
                'chunk_type': meta.get('chunk_type'),
                'spec_categorie': meta.get('spec_category'),
                'product_id': meta.get('product_id'),
                'product_manufacturer': meta.get('product_manufacturer')
            })

    return pd.DataFrame(results)

In [ ]:
df = retrieve(all_queries)

df.to_json('retrivals.json', orient='records', lines=True, force_ascii=False)

# print(df)

## Metriken

### Euclidean Distance

Zeigt wie gut die Retivals sind.

L2-Distanz (Euklidean Distance) zwischen Query und Retrival Chunk im normalisierten Vektorspace. Je kleiner desto besser (0 = identisch, ~0,2 = sehr ähnlich, ...). 

- Ranking: Statistiken pro Rang
- Distance Gap: Abstand zwischen ersten und zehnten Rang
- Strong Results: Erbenisse mit einer Similarity > 0,3

In [ ]:
distance_stats = df.groupby('rank')['distance'].agg(['mean','median','std','min','max'])
print("Ranking")
print(distance_stats)
print("\n")

rank01_mean = df[df['rank'] == 1]['distance'].mean()
rank10_mean = df[df['rank'] == 10]['distance'].mean()
gap = rank10_mean - rank01_mean
print(f"Distance Gap (Rank 1 -> 10): {gap:.4f}")

# Cousin Sim
high_confidence = (df['distance'] < 0.3).sum()
total = len(df)
print(f"Strong Results: {high_confidence}/{total} ({high_confidence/total*100:.1f}%)")


#### Cousin Similarity und Dot Product

Die Cousin Similarity berechnet den Winkel zweier Vektoren. Nie Länge ist hier irrelevant.

Selbes zeigt auch das Dot Product, auch hier wird verglichen ob die Vektoren in die selbe Richtung zeigen.

### Consistency & Stability

- Tied Sum: Anzahl der Results mit gleicher Distanz -> Führtdazu, dass das Model nicht gut unterscheiden kann.

In [ ]:
duplicate_distances = df.groupby(['query', 'distance']).size()
ties = (duplicate_distances > 1).sum()
print(f"Tied Sum: {ties}")

### Coverage

Anzahl der unterschiedlichen Chunks über alle Queries -> Zeigt die Abdeckung der Daten in der DB - oder ob immer die selben Chunks gefunden werden.

- Unique Chunks: Anzahl der Chunks sie einmalig gefunden wurden
- Most Wanted: Eigentlich die am häufigsten gefundenen Chunks

In [ ]:
unique_chunks = df['document'].nunique()
total_chunks_in_db = collection.count()
coverage = unique_chunks / total_chunks_in_db * 100
print(f"Unique Chunks: {unique_chunks}/{total_chunks_in_db} ({coverage:.1f}%)")
print("\n")

chunk_frequency = df['document'].value_counts()
most_common = chunk_frequency.head(5)
print("Most Wanted")
for doc, count in most_common.items():
    print(f"{count}x: {doc[:100]}...")
print("\n")

#### Chunk-Type Analysis

In [ ]:
print(df.groupby(by=['chunk_type']).sum())

#### Cross-Encoding

Ich möchte zudem vesuchen mit einem Cross-Encoder zu evaluieren.